In [1]:
devtools::install_github("clairbarnes/rwwa")
library(rwwa)

Skipping install of 'rwwa' from a github remote, the SHA1 (d52021fc) has not changed since last install.
  Use `force = TRUE` to force installation

Loading required package: extRemes

Loading required package: Lmoments

Loading required package: distillery


Attaching package: ‘extRemes’


The following objects are masked from ‘package:stats’:

    qqnorm, qqplot


Loading required package: plyr


Attaching package: ‘plyr’


The following object is masked from ‘package:distillery’:

    is.formula




# Trend fitting for climate models

## CORDEX

### GMST only

In [14]:
for (fnm in list.files("ts", pattern = "wetseason_.AM-[0-9]{2}", full.names = T)) {
    
    res_fnm <- gsub(".dat", ".csv", gsub("ts/", "res/", fnm))
    if(!file.exists(res_fnm)) {
        ts <- load_ts(fnm, col.names = c("year", "pr"))
        gmst <- load_ts(list.files("ts", pattern = paste0("smoothed-gsat_", strsplit(fnm, "_")[[1]][4]), full.names = T), col.names = c("year", "gmst"))

        df <- merge(ts, gmst)

        cov_2023 <- df[df$year == 2023, "gmst", drop = F]
        cov_hist <- df[df$year == 2023, "gmst", drop = F] - 1.2
        cov_fut <- df[df$year == 2023, "gmst", drop = F] + 0.8

        mdl <- fit_ns("norm", "fixeddisp", df, "pr", "gmst", lower = T)

        res <- cmodel_results(mdl, rp = 10, cov_f = cov_2023, cov_hist = cov_hist, cov_fut = cov_fut,
                             y_now = 2023, y_start = 1979, y_fut = 2050, nsamp = 500)

        write.csv(res, res_fnm)
    }
}

### GMST + Nino3.4

In [2]:
rp = 10
nsamp = 1000

# use 2023 observed value of Nino3.4 as factual covariate
nino_obs <- load_ts("ts/panama_nino-rs.dat", col.names = c("year", "nino"))
nino_2023 <- nino_obs$nino[nino_obs$year == 2023]

# currently no Nino data for EC-EARTH
fl <- list.files("ts", pattern = "wetseason_.AM-[0-9]{2}", full.names = T)
fl <- fl[!grepl("EC-EARTH", fl)]

In [6]:
for (fnm in fl) {
    
    nino_fnm <- list.files("ts", pattern = paste0("nino-rs_", paste0(strsplit(fnm, "_")[[1]][4:5], collapse = "_")), full.names = T)
    if(length(nino_fnm) == 0) next

    # load data & fit model
    ts <- load_ts(fnm, col.names = c("year", "pr"))
    gmst <- load_ts(list.files("ts", pattern = paste0("smoothed-gsat_", strsplit(fnm, "_")[[1]][4]), full.names = T), col.names = c("year", "gmst"))
    nino <- load_ts(nino_fnm, col.names = c("year", "nino"))
    df <- merge(ts, merge(gmst, nino))
    
    mdl <- fit_ns("norm", "fixeddisp", df, "pr", c("gmst", "nino"), lower = T)

    res_fnm <- gsub("pr-wetseason", "pr-wetseason-nino", gsub(".dat", ".csv", gsub("ts/", "res/", fnm)))
    if(!file.exists(res_fnm)) {
        
        # use model's 2023 GMST & observed Nino to define factual climate
        gmst_2023 <- df$gmst[df$year == 2023]
        
        cov_2023 <- data.frame(gmst = gmst_2023, nino = nino_2023)
        cov_cf <- rbind("pi" = data.frame("gmst" = gmst_2023 - 1.2, "nino" = nino_2023), 
                        "neut" = data.frame("gmst" = gmst_2023, "nino" = 0))
        cov_fut <- data.frame(gmst = gmst_2023 + 0.8, nino = nino_2023)

        # bootstrap results
        res <- cmodel_results(mdl, rp = rp, cov_f = cov_2023, cov_hist = cov_cf, cov_fut = cov_fut,
                             y_now = 2023, y_start = 1979, y_fut = 2050, nsamp = nsamp)
        
        # bootstrap correlation between Nino & precip in this region
        set.seed(42)
        nino_corr <- c(cor(df$nino, df$pr), 
                       quantile(sapply(1:nsamp, function(i) cor(df[sample(1:nrow(mdl$data), replace = T), c("nino", "pr")])[1,2]), c(0.025, 0.975)))
        nino_corr <- setNames(data.frame(t(nino_corr)), paste0("nino-corr_", c("est", "lower", "upper")))
                                       
        res <- cbind(nino_corr, res)
        write.csv(res, res_fnm)
    }
                                       
    # quick return level plots to check fit
    rlplot_fnm <- gsub("csv", "png", gsub("res/", "mdl-eval/rlplot_", res_fnm))
    if(!file.exists(rlplot_fnm)) {
        png(rlplot_fnm, h = 480, w = 480 * 2); par(cex.main = 1.4, mfrow = c(1,2), oma = c(0,0,2,0)); {
            plot_returnlevels(mdl, cov_f = cov_2023, cov_cf = cov_cf["pi",,drop = F], nsamp = 100, main = "Present vs PI")
            plot_returnlevels(mdl, cov_f = cov_2023, cov_cf = cov_cf["neut",,drop = F], nsamp = 100, main ="Present vs neutral Nino3.4")
            mtext(outer = T, gsub(".dat","",paste0(strsplit(fnm, "_")[[1]][4:6], collapse = "_")), font = 2, cex = 1.5)
        }; dev.off()
    }
}

In [27]:
# compile results
res <- t(sapply(list.files("res", pattern = "pr-wetseason-nino_.AM-[0-9]{2}", full.names = T), read.csv, row.names = "X"))
rownames(res) <- sapply(rownames(res), function(rnm) gsub(".csv", "", paste0(strsplit(rnm, "_")[[1]][3:6], collapse = "_")))

In [29]:
write.csv(res, "res_pr-wetseason-nino_cordex.csv")